<a href="https://colab.research.google.com/github/taniokah/DL-Basic-Seminar/blob/master/AI%E3%83%96%E3%83%BC%E3%83%A0%E5%88%86%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#AIブーム分析

https://ohke.hateblo.jp/entry/2019/06/01/120000
https://qiita.com/a2kiti/items/a93d762bd89a84316c12

1. まず、Sudachiのword2vecモデルをダウンロードする
2. 対象となる新聞記事を読み込む
3. 新聞記事からAIに近い単語を上位10個ずつ抽出する
4. 単語ベクトルとその頻度のリストを作る
5. 時系列トピックモデルを用いて分析する


In [1]:
!wget https://object-storage.tyo2.conoha.io/v1/nc_2520839e1f9641b08211a5c85243124a/word_vector/nwjc.sudachi_full_abc_w2v.20190314.tar.gz

--2020-01-28 10:50:54--  https://object-storage.tyo2.conoha.io/v1/nc_2520839e1f9641b08211a5c85243124a/word_vector/nwjc.sudachi_full_abc_w2v.20190314.tar.gz
Resolving object-storage.tyo2.conoha.io (object-storage.tyo2.conoha.io)... 157.7.224.17
Connecting to object-storage.tyo2.conoha.io (object-storage.tyo2.conoha.io)|157.7.224.17|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5212195768 (4.9G) [application/x-www-form-urlencoded]
Saving to: ‘nwjc.sudachi_full_abc_w2v.20190314.tar.gz’

nwjc.sudachi_full_a 100%[===================>]   4.85G  19.1MB/s    in 4m 27s  

2020-01-28 10:55:22 (18.6 MB/s) - ‘nwjc.sudachi_full_abc_w2v.20190314.tar.gz’ saved [5212195768/5212195768]



In [2]:
!tar -zxvf ./nwjc.sudachi_full_abc_w2v.20190314.tar.gz

nwjc_sudachi_full_abc_w2v/
nwjc_sudachi_full_abc_w2v/nwjc.sudachi_full_abc_w2v.txt
nwjc_sudachi_full_abc_w2v/LICENSE


In [3]:
!ls -l nwjc_sudachi_full_abc_w2v/

total 12229548
-rw-rw-r-- 1 1008 1008       11420 Mar 20  2019 LICENSE
-rw-rw-r-- 1 1008 1008 12523039651 Mar 20  2019 nwjc.sudachi_full_abc_w2v.txt


In [4]:
!head -n2 nwjc_sudachi_full_abc_w2v/nwjc.sudachi_full_abc_w2v.txt

3644628 300
の -0.08509623 -0.02170456 -0.00595318 0.17987297 -0.02470185 -0.06039717 0.16489118 0.18576832 -0.13870764 -0.17454691 0.08653757 -0.11530939 -0.0357993 0.0308698 -0.11082912 0.04128339 0.00738431 0.00342017 -0.00673456 0.03348965 -0.18876095 0.05184764 0.13508447 0.24196455 -0.07351201 -0.07903086 -0.01973874 -0.24246444 0.03141621 -0.11729659 -0.04126292 -0.20383213 -0.03109288 0.06765312 -0.15020953 -0.11498244 0.03988069 0.20287995 -0.14308368 -0.05135961 -0.15592268 0.05364651 -0.11668575 0.0505892 -0.04575419 0.03296537 -0.03033004 -0.0481239 0.13578212 -0.03111291 -0.15257519 -0.10075521 -0.07440718 -0.12524064 -0.14464405 -0.00155862 0.11825287 0.01658496 0.27135071 -0.07724451 0.01130517 0.03584538 0.01659033 0.19819288 0.01558947 0.07239497 -0.05149494 0.08688559 -0.03712051 -0.07294089 -0.00820937 -0.00907754 -0.03328617 -0.09273465 0.07537545 -0.05137397 0.07578564 -0.03124459 0.05484971 -0.07136369 -0.02457538 0.15481928 0.11072636 -0.00225826 -0.04312619 -0.09

In [0]:
#!pip install gensim

In [6]:
# モデルのロード
from gensim.models import KeyedVectors
from gensim.test.utils import datapath

nwjc_model = KeyedVectors.load_word2vec_format('/content/nwjc_sudachi_full_abc_w2v/nwjc.sudachi_full_abc_w2v.txt', binary=False)

#nwjc_model = KeyedVectors.load_word2vec_format(
#    datapath('/content/nwjc_sudachi_full_abc_w2v/nwjc.sudachi_full_abc_w2v.txt'),
#    binary=False
#)

# 語数, 次元数
print(len(nwjc_model.vocab), nwjc_model.vector_size)  # 3644628 300

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


3644628 300


In [7]:
print(nwjc_model.most_similar('平成', topn=5))
# [('年度', 0.8092531561851501),
#  ('平成元年', 0.7409119009971619),
#  ('同年', 0.6042426824569702),
#  ('基本計画', 0.5917632579803467),
#  ('別紙', 0.590812087059021)]

print(nwjc_model.most_similar(positive=['兄弟', '女'], negative=['男'], topn=5))
# [('姉妹', 0.7687463164329529),
#  ('兄妹', 0.7484474182128906),
#  ('姉弟', 0.715774416923523),
#  ('双子', 0.642593264579773),
#  ('妹', 0.6402267217636108)]

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('年度', 0.8092531561851501), ('平成元年', 0.7409119009971619), ('同年', 0.6042426824569702), ('基本計画', 0.5917632579803467), ('別紙', 0.590812087059021)]
[('姉妹', 0.7687463164329529), ('兄妹', 0.7484474182128906), ('姉弟', 0.715774416923523), ('双子', 0.642593264579773), ('妹', 0.6402267217636108)]


In [9]:
!pip install SudachiPy

     |████████████████████████████████| 81kB 2.3MB/s 
  Created wheel for dartsclone: filename=dartsclone-0.6-cp36-cp36m-linux_x86_64.whl size=413254 sha256=c799b83e077e3d2ba16a48bbfb88ece61ed0b85d6ab9f3e3644ead2457e5b845
  Stored in directory: /root/.cache/pip/wheels/be/cd/70/fe43307bf7398243155108f4f5a258ef336923d65ec4af93cd
Successfully built dartsclone


In [10]:
!pip install https://object-storage.tyo2.conoha.io/v1/nc_2520839e1f9641b08211a5c85243124a/sudachi/SudachiDict_core-20191224.tar.gz

     |████████████████████████████████| 70.7MB 60kB/s 
  Created wheel for SudachiDict-core: filename=SudachiDict_core-20191224-cp36-none-any.whl size=70876855 sha256=094191d18163296345a8522c3a480ba094c99f3b57026b028ddfc06f447071ab
  Stored in directory: /root/.cache/pip/wheels/4c/88/5c/44c0deb66c147b881dab3fe061febbb301bdceab7e9b496479
Successfully built SudachiDict-core


In [12]:
!sudachipy tokenize -h

usage: sudachipy tokenize [-h] [-r file] [-m {A,B,C}] [-o file] [-a] [-d] [-v]
                          [file [file ...]]

Tokenize Text

positional arguments:
  file           text written in utf-8

optional arguments:
  -h, --help     show this help message and exit
  -r file        the setting file in JSON format
  -m {A,B,C}     the mode of splitting
  -o file        the output file
  -a             print all of the fields
  -d             print the debug information
  -v, --version  print sudachipy version


In [0]:
import numpy as np
import pickle
import re
import csv
import os
from sudachipy import tokenizer
from sudachipy import dictionary

class Sudachi_w2v:
    def __init__(self, path, sudachiDataPath="sudachiData.pickle"):
        f = open(path, 'r')
        self.file = f
        self.reader = csv.reader(f, delimiter=' ')
        #最初に含有単語リストやメモリアドレスリストを作成する（かなり時間かかる）
        #2回目以降はpickle化したものを読み込む
        if os.path.exists(sudachiDataPath):
            with open(sudachiDataPath, 'rb') as f:
                dataset = pickle.load(f)
            self.offset_list = dataset["offset_list"]
            self.emb_size = dataset["emb_size"]
            self.word2index = dataset["word2index"]
            self.ave_vec = dataset["ave_vec"]
        else:
            txt = f.readline()
            #分散表現の次元数
            self.emb_size = int(txt.split()[1])
            #未知語が来た場合平均ベクトルを返す
            self.ave_vec = np.zeros(self.emb_size, np.float)
            #メモリアドレスリスト
            self.offset_list = []
            word_list = []
            count = 0
            maxCount = int(txt.split()[0])
            while True:
                count+=1
                self.offset_list.append(f.tell())
                if count % 100000 == 0:print(count,"/",maxCount)
                line = f.readline()
                if line == '':break
                line_list = line.split()
                word_list.append(line_list[0])
                self.ave_vec += np.array(line_list[-300:]).astype(np.float)
            self.offset_list.pop()
            self.ave_vec = self.ave_vec/count
            self.word2index = {v:k for k,v in enumerate(word_list)}

            dataset = {}
            dataset["offset_list"] = self.offset_list
            dataset["emb_size"] = self.emb_size
            dataset["word2index"] = self.word2index
            dataset["ave_vec"] = self.ave_vec
            with open(sudachiDataPath, 'wb') as f:
                pickle.dump(dataset, f)

        self.num_rows = len(self.offset_list)
        #sudachiの準備
        self.tokenizer_obj = dictionary.Dictionary().create()
        self.mode = tokenizer.Tokenizer.SplitMode.C

    #単語をベクトル化
    def word2vec(self, word):
        try:
            idx = self.word2index[word]
            result = self.read_row(idx)
            vec = np.array(result[-300:])
            return vec
        except:#単語リストにない場合
            print(word, ": out of wordlist")

    #文章を分かち書きした後，それぞれのベクトルをmatでまとめて返す
    def sentence2mat(self, sentence):
        words = sentence.replace("　"," ").replace("\n"," ")
        words = re.sub(r"\s+", " ", words)
        input_seq = [m.surface().lower() for m in self.tokenizer_obj.tokenize(words, self.mode)]
        input_seq = [s for s in input_seq if s != ' ']

        mat = np.zeros((len(input_seq), self.emb_size))
        input_sentence = []
        for i, word in enumerate(input_seq):
            try:
                idx = self.word2index[word]
                result = self.read_row(idx)
                input_sentence.append(result[0])
                mat[i] = np.array(result[-300:])
            except:#単語リストにない場合平均ベクトルを返す
                input_sentence.append("<UNK>")
                mat[i] = self.ave_vec
        return input_sentence, mat

    def __del__(self):
        self.file.close()

    def read_row(self, idx):
        self.file.seek(self.offset_list[idx])
        return next(self.reader)

In [0]:
path = '/content/nwjc_sudachi_full_abc_w2v/nwjc.sudachi_full_abc_w2v.txt'
sudachi = Sudachi_w2v(path)

In [16]:
vec = sudachi.word2vec("すだち")
print(vec)
#['0.07975651' '0.08931299' '-0.06070593' '0.46959993' '0.19651023' ~

input_sentence, mat = sudachi.sentence2mat("あきらめたらそこで試合終了だよ")
print(input_sentence)
print(mat)
#(['あきらめ', 'たら', 'そこ', 'で', '試合終了', 'だ', 'よ'], array([[ 1.9788130e-02,  1.1190426e-01, -1.6153505e-01, ...,

input_sentence, mat = sudachi.sentence2mat("AIってなんだろ？")
print(input_sentence)
print(mat)


['0.07975651' '0.08931299' '-0.06070593' '0.46959993' '0.19651023'
 '0.05610859' '-0.24515596' '-0.30732957' '0.08137251' '-0.45044619'
 '0.11408115' '-0.05047991' '-0.06369073' '-0.30423751' '-0.05283662'
 '-0.12789412' '-0.08447175' '0.18121149' '0.07306494' '-0.34626028'
 '-0.23872021' '-0.22532608' '0.08475439' '0.03715251' '-0.11880537'
 '-0.16653691' '0.01077076' '0.21094765' '-0.14410816' '-0.05907292'
 '-0.17079048' '-0.10630048' '0.42154697' '0.19493569' '0.05778833'
 '-0.02867538' '-0.03691955' '0.2445851' '-0.41814491' '-0.01401133'
 '-0.26229978' '0.01251183' '0.02726642' '0.03536683' '0.0740893'
 '-0.36967111' '-0.02937168' '-0.29803264' '-0.1353' '-0.03620005'
 '-0.03665653' '0.28185824' '-0.43465024' '0.09290418' '-0.15962832'
 '0.07909478' '-0.28243098' '0.15427014' '0.07028364' '0.12094842'
 '-0.47612959' '-0.26188281' '0.14339833' '-0.0164896' '0.07793605'
 '-0.00731504' '0.17469509' '-0.15381725' '-0.13285433' '-0.1341684'
 '0.04934204' '0.21530381' '0.23320644' '0.1